# CALFEM

> Fill in a module description here

In [ ]:
# | default_exp _calfem

In [ ]:
# | export

# Copyright © 2023-2024  IfcTruss Contributors
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# | hide
import nbdev
import nbdev.showdoc

In [ ]:
from rich import print

In [ ]:
# | export
from fastcore.basics import patch
import numpy as np
import pandas as pd

In [ ]:
# | export
try:
    import calfem.core as cfc
except ImportError:
    pass

In [ ]:
# | export
class CALFEMException(Exception):
    pass

In [ ]:
# | export
class CALFEM:
    def __init__(
        self,
        *,
        bars,
        nodes,
        point_loads,
    ):
        # copied from DirectStiffnessMethod class
        self.bars = bars
        self.nodes = nodes
        self.point_loads = point_loads

In [ ]:
Nodes_data = {
    "Node": pd.Series([2, 1, 3, 4], dtype=int),
    "Coordinate_X": pd.Series([0, 0, -4e3, -4e3], dtype=float),
    "Coordinate_Y": pd.Series([0, 0, 0, 0], dtype=float),
    "Coordinate_Z": pd.Series([3e3, 0, 3e3, 6e3], dtype=float),
    "Translational_X": pd.Series([0, 1, 1, 1], dtype=bool),
    "Translational_Y": pd.Series([1, 1, 1, 1], dtype=bool),
    "Translational_Z": pd.Series([0, 1, 1, 1], dtype=bool),
}

Nodes = pd.DataFrame(Nodes_data)
Nodes.style.hide(axis="index")

Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z
2,0.000000,0.000000,3000.000000,False,True,False
1,0.000000,0.000000,0.000000,True,True,True
3,-4000.000000,0.000000,3000.000000,True,True,True
4,-4000.000000,0.000000,6000.000000,True,True,True


In [ ]:
Bars_data = {
    "Bar": pd.Series([1, 2, 3], dtype=int),
    "Start_node": pd.Series([2, 2, 2], dtype=int),
    "End_node": pd.Series([1, 3, 4], dtype=int),
    "Cross-sectional_area": pd.Series([1e3, 1e3, 1e3], dtype=float),
    "Modulus_of_elasticity": pd.Series([1e3, 1e3, 1e3], dtype=float),
}
Bars = pd.DataFrame(Bars_data)
Bars.style.hide(axis="index")

Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity
1,2,1,1000.000000,1000.000000
2,2,3,1000.000000,1000.000000
3,2,4,1000.000000,1000.000000


In [ ]:
Point_Loads_data = {
    "Point_Load": pd.Series(
        [
            1,
            2,
            3,
        ],
        dtype=int,
    ),
    "Node": pd.Series(
        [2, 4, 4],
        dtype=int,
    ),
    "Force_X": pd.Series(
        [100e3, 100e3, 50e3],
        dtype=float,
    ),
    "Force_Y": pd.Series(
        [0, 0, 0],
        dtype=float,
    ),
    "Force_Z": pd.Series(
        [-100e3, 0, 50e3],
        dtype=float,
    ),
}
Point_Loads = pd.DataFrame(Point_Loads_data)
Point_Loads.style.hide(axis="index")

Point_Load,Node,Force_X,Force_Y,Force_Z
1,2,100000.000000,0.000000,-100000.000000
2,4,100000.000000,0.000000,0.000000
3,4,50000.000000,0.000000,50000.000000


In [ ]:
System = CALFEM(
    bars=Bars,
    nodes=Nodes,
    point_loads=Point_Loads,
)

In [ ]:
# | export
@patch
def extend_nodes_df(
    self: CALFEM,
):
    # copied from DirectStiffnessMethod class
    self.number_of_rows = self.nodes.shape[0]
    self.dimensions = 3

    self.number_of_degrees_of_freedom = (
        self.number_of_rows * self.dimensions
    )

    self.nodes["Degrees_of_freedom"] = pd.Series(
        np.split(
            np.arange(self.number_of_degrees_of_freedom),
            self.number_of_rows,
        )
    )

In [ ]:
System.extend_nodes_df()

In [ ]:
System.nodes.style.hide(axis="index")

Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z,Degrees_of_freedom
2,0.000000,0.000000,3000.000000,False,True,False,[0 1 2]
1,0.000000,0.000000,0.000000,True,True,True,[3 4 5]
3,-4000.000000,0.000000,3000.000000,True,True,True,[6 7 8]
4,-4000.000000,0.000000,6000.000000,True,True,True,[ 9 10 11]


In [ ]:
# | export
@patch
def extend_bars_df(
    self: CALFEM,
):
    # copied from DirectStiffnessMethod class
    start_node = self.nodes.rename(
        columns={
            "Node": "Start_node",
            "Coordinate_X": "x_1",
            "Coordinate_Y": "y_1",
            "Coordinate_Z": "z_1",
            "Degrees_of_freedom": "Degrees_of_freedom_1",
        }
    )

    bars_start = pd.merge(
        self.bars,
        start_node[
            ["Start_node", "x_1", "y_1", "z_1", "Degrees_of_freedom_1"]
        ],
        on="Start_node",
    )

    end_node = self.nodes.rename(
        columns={
            "Node": "End_node",
            "Coordinate_X": "x_2",
            "Coordinate_Y": "y_2",
            "Coordinate_Z": "z_2",
            "Degrees_of_freedom": "Degrees_of_freedom_2",
        }
    )

    self.bars_extended = pd.merge(
        bars_start,
        end_node[["End_node", "x_2", "y_2", "z_2", "Degrees_of_freedom_2"]],
        on="End_node",
    )

In [ ]:
System.extend_bars_df()

In [ ]:
System.bars_extended.style.hide(axis="index")

Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity,x_1,y_1,z_1,Degrees_of_freedom_1,x_2,y_2,z_2,Degrees_of_freedom_2
1,2,1,1000.000000,1000.000000,0.000000,0.000000,3000.000000,[0 1 2],0.000000,0.000000,0.000000,[3 4 5]
2,2,3,1000.000000,1000.000000,0.000000,0.000000,3000.000000,[0 1 2],-4000.000000,0.000000,3000.000000,[6 7 8]
3,2,4,1000.000000,1000.000000,0.000000,0.000000,3000.000000,[0 1 2],-4000.000000,0.000000,6000.000000,[ 9 10 11]


In [ ]:
# | export
@patch
def create_element_stiffness_matrice(self: CALFEM, row):
    ex = row[["x_1", "x_2"]]
    ey = row[["y_1", "y_2"]]
    ez = row[["z_1", "z_2"]]
    ep = [row["Modulus_of_elasticity"], row["Cross-sectional_area"]]
    K_e = cfc.bar3e(ex, ey, ez, ep)

    return K_e

In [ ]:
# | export
@patch
def create_element_stiffness_matrices(
    self: CALFEM,
):
    self.bars_extended["Element_Matrice"] = self.bars_extended.apply(
        self.create_element_stiffness_matrice, axis=1
    )

    self.bars_extended["Degrees_of_freedom"] = self.bars_extended.apply(
        lambda row: np.concatenate(
            (row["Degrees_of_freedom_1"], row["Degrees_of_freedom_2"])
        ),
        axis=1,
    )

    self.element_stiffness_matrices_df = self.bars_extended[
        ["Bar", "Element_Matrice", "Degrees_of_freedom"]
    ]

In [ ]:
System.bars_extended[["x_1", "x_2"]].values

array([[    0.,     0.],
       [    0., -4000.],
       [    0., -4000.]])

In [ ]:
System.create_element_stiffness_matrices()

In [ ]:
System.element_stiffness_matrices_df

,Bar,Element_Matrice,Degrees_of_freedom
0,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....","[0, 1, 2, 3, 4, 5]"
1,2,"[[250.0, 0.0, 0.0, -250.0, 0.0, 0.0], [0.0, 0....","[0, 1, 2, 6, 7, 8]"
2,3,"[[128.0, 0.0, -96.0, -128.0, 0.0, 96.0], [0.0,...","[0, 1, 2, 9, 10, 11]"


In [ ]:
# | export
@patch
def create_system_stiffness_matrice(
    self: CALFEM,
):
    self.K_f = np.zeros(
        (
            self.number_of_degrees_of_freedom,
            self.number_of_degrees_of_freedom,
        )
    )

    for element_matrice, degrees_of_freedom in zip(
        self.element_stiffness_matrices_df["Element_Matrice"],
        self.element_stiffness_matrices_df["Degrees_of_freedom"],
    ):
        cfc.assem(degrees_of_freedom + 1, self.K_f, element_matrice)

In [ ]:
System.create_system_stiffness_matrice()

In [ ]:
print(System.K_f)

[[ 378.            0.          -96.            0.            0.
     0.         -250.            0.            0.         -128.
     0.           96.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [ -96.            0.          405.33333333    0.            0.
  -333.33333333    0.            0.            0.           96.
     0.          -72.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.         -333.33333333    0.            0.
   333.33333333    0.            0.            0.            0.
     0.            0.        ]
 [-250.            0.            0.            0.            0.
     0.          250.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [-128.            0.           96.            0.            0.
     0.            0.            0.            0.          128.
     0.          -96.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [  96.            0.          -72.            0.            0.
     0.            0.            0.            0.          -96.
     0.           72.        ]]

In [ ]:
# | export
@patch
def create_force_vector(
    self: CALFEM,
):
    # copied from DirectStiffnessMethod class
    self.F_f = np.zeros(self.number_of_degrees_of_freedom)

    self.point_loads_extended = pd.merge(
        self.point_loads,
        self.nodes[["Node", "Degrees_of_freedom"]],
        on="Node",
    )

    np.add.at(
        self.F_f,
        np.concatenate(
            self.point_loads_extended["Degrees_of_freedom"].values
        ),
        np.concatenate(
            self.point_loads_extended[
                ["Force_X", "Force_Y", "Force_Z"]
            ].values
        ),
    )

In [ ]:
System.create_force_vector()

In [ ]:
System.F_f

array([ 100000.,       0., -100000.,       0.,       0.,       0.,
             0.,       0.,       0.,  150000.,       0.,   50000.])

In [ ]:
# | export
@patch
def calculate_displacement_vector_and_calculate_force_vector(
    self: CALFEM,
):
    self.degrees_of_freedom = (
        np.concatenate(
            self.nodes[
                [
                    "Degrees_of_freedom",
                ]
            ].values.flatten()
        )
        + 1
    )

    self.translationals = np.logical_not(
        self.nodes[
            [
                "Translational_X",
                "Translational_Y",
                "Translational_Z",
            ]
        ].values.flatten()
    )
    self.bc = self.degrees_of_freedom[~self.translationals]

    # Point Load in support are zero for the following calculation
    F = self.F_f.copy()
    F[~self.translationals] = 0

    try:
        self.u_f, self.F_f_calculated = cfc.solveq(
            self.K_f,
            F.reshape((self.number_of_degrees_of_freedom, 1)),
            self.bc,
        )
    except np.linalg.LinAlgError as l:
        if str(l) == "Singular matrix":
            raise CALFEMException("Mechanism - The system is unstable")
        else:
            raise l
    self.u_f = np.array(self.u_f).reshape(self.number_of_degrees_of_freedom)

    self.F_f_calculated = np.array(self.F_f_calculated).reshape(
        self.number_of_degrees_of_freedom
    )

In [ ]:
System.calculate_displacement_vector_and_calculate_force_vector()

In [ ]:
System.bc

array([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [ ]:
System.u_f

array([ 214.81481481,    0.        , -195.83333333,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

In [ ]:
System.F_f_calculated

array([ 0.00000000e+00,  0.00000000e+00,  1.45519152e-11,  0.00000000e+00,
        0.00000000e+00,  6.52777778e+04, -5.37037037e+04,  0.00000000e+00,
        0.00000000e+00, -4.62962963e+04,  0.00000000e+00,  3.47222222e+04])

If the user has defined a point load in the support, this force must be considered in the support forces. This is achieved with the function `correct_force_vector`.

In [ ]:
# | export
@patch
def correct_force_vector(
    self: CALFEM,
):
    self.F_f_nodes = self.F_f.copy()
    self.F_f_nodes[self.translationals] = -self.F_f_nodes[
        self.translationals
    ]

    self.F_f_corrected = self.F_f_calculated - self.F_f_nodes

In [ ]:
System.correct_force_vector()

In [ ]:
System.F_f_calculated

array([ 0.00000000e+00,  0.00000000e+00,  1.45519152e-11,  0.00000000e+00,
        0.00000000e+00,  6.52777778e+04, -5.37037037e+04,  0.00000000e+00,
        0.00000000e+00, -4.62962963e+04,  0.00000000e+00,  3.47222222e+04])

In [ ]:
System.F_f_corrected

array([ 100000.        ,       0.        , -100000.        ,
             0.        ,       0.        ,   65277.77777778,
        -53703.7037037 ,       0.        ,       0.        ,
       -196296.2962963 ,       0.        ,  -15277.77777778])

In [ ]:
# | export
@patch
def calculate_normal_force_in_df(self: CALFEM, row):
    ex = row[["x_1", "x_2"]]
    ey = row[["y_1", "y_2"]]
    ez = row[["z_1", "z_2"]]
    ep = [row["Modulus_of_elasticity"], row["Cross-sectional_area"]]
    degrees_of_freedom = row["Degrees_of_freedom"]
    ed = cfc.extractEldisp(
        degrees_of_freedom + 1,
        self.u_f.reshape((self.number_of_degrees_of_freedom, 1)),
    )

    N = cfc.bar3s(ex, ey, ez, ep, ed)
    return N[0][0]

In [ ]:
# | export
@patch
def calculate_normal_force(self: CALFEM):
    # copied from DirectStiffnessMethod class
    self.bars_extended["Normal_force"] = self.bars_extended.apply(
        self.calculate_normal_force_in_df, axis=1
    )

In [ ]:
System.calculate_normal_force()

In [ ]:
System.bars_extended

,Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity,x_1,y_1,z_1,Degrees_of_freedom_1,x_2,y_2,z_2,Degrees_of_freedom_2,Element_Matrice,Degrees_of_freedom,Normal_force
0,1,2,1,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",0.0,0.0,0.0,"[3, 4, 5]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....","[0, 1, 2, 3, 4, 5]",-65277.777778
1,2,2,3,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",-4000.0,0.0,3000.0,"[6, 7, 8]","[[250.0, 0.0, 0.0, -250.0, 0.0, 0.0], [0.0, 0....","[0, 1, 2, 6, 7, 8]",53703.703704
2,3,2,4,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",-4000.0,0.0,6000.0,"[9, 10, 11]","[[128.0, 0.0, -96.0, -128.0, 0.0, 96.0], [0.0,...","[0, 1, 2, 9, 10, 11]",57870.370370


In [ ]:
System.bars_extended["Normal_force"]

0   -65277.777778
1    53703.703704
2    57870.370370
Name: Normal_force, dtype: float64

In [ ]:
System.u_f

array([ 214.81481481,    0.        , -195.83333333,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

In [ ]:
System.nodes

,Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z,Degrees_of_freedom
0,2,0.0,0.0,3000.0,False,True,False,"[0, 1, 2]"
1,1,0.0,0.0,0.0,True,True,True,"[3, 4, 5]"
2,3,-4000.0,0.0,3000.0,True,True,True,"[6, 7, 8]"
3,4,-4000.0,0.0,6000.0,True,True,True,"[9, 10, 11]"


In [ ]:
# | export
@patch
def create_displacment_df(self: CALFEM):
    # copied from DirectStiffnessMethod class
    self.dimensions_names = ["X", "Y", "Z"]
    for index, name in enumerate(self.dimensions_names):
        self.nodes[f"Displacement_{name}"] = self.u_f[
            index :: self.dimensions
        ]

    self.translational_names = [
        "Translational_X",
        "Translational_X",
        "Translational_X",
    ]
    self.displacment_df = self.nodes[
        ~(self.nodes[self.translational_names].all(axis=1))
    ]

    self.displacment_df = self.displacment_df[
        ["Node", "Displacement_X", "Displacement_Y", "Displacement_Z"]
    ]

In [ ]:
System.create_displacment_df()

In [ ]:
System.displacment_df.style.hide(axis="index")

Node,Displacement_X,Displacement_Y,Displacement_Z
2,214.814815,0.000000,-195.833333


In [ ]:
# | export
@patch
def create_force_df(self: CALFEM):
    # copied from DirectStiffnessMethod class
    for index, name in enumerate(self.dimensions_names):
        self.nodes[f"Force_{name}"] = self.F_f_corrected[
            index :: self.dimensions
        ]

    self.force_df = self.nodes[
        (self.nodes[self.translational_names].all(axis=1))
    ]

    self.force_df = self.force_df[["Node", "Force_X", "Force_Y", "Force_Z"]]

In [ ]:
System.create_force_df()

In [ ]:
System.force_df.style.hide(axis="index")

Node,Force_X,Force_Y,Force_Z
1,0.000000,0.000000,65277.777778
3,-53703.703704,0.000000,0.000000
4,-196296.296296,0.000000,-15277.777778


In [ ]:
# | export
@patch
def create_normal_force_df(self: CALFEM):
    # copied from DirectStiffnessMethod class
    self.normal_force_df = self.bars_extended[
        ["Bar", "Normal_force"]
    ].copy()
    self.normal_force_df["Type_of_normal_force"] = self.normal_force_df[
        "Normal_force"
    ].apply(
        lambda x: (
            "Tensile force"
            if x > 0
            else ("Compressive force" if x < 0 else "Zero-force")
        )
    )

In [ ]:
System.create_normal_force_df()

In [ ]:
System.normal_force_df.style.hide(axis="index")

Bar,Normal_force,Type_of_normal_force
1,-65277.777778,Compressive force
2,53703.703704,Tensile force
3,57870.370370,Tensile force


In [ ]:
# | hide
nbdev.nbdev_export()